<a href="https://colab.research.google.com/github/RafsanJany-44/ARC/blob/master/Primarly_VitalDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
# Importing Numpy module
import numpy as np

# Creating a 2-D Numpy array
n_arr = np.array([[45.5, 52, 10],
                  [5, 50, 25]])
print("Given array:")
print(n_arr)

print("\nReplace all elements of array which are greater than 30. to 5.25")
n_arr[0][1] = n_arr[0][0]

print("New array :\n")
print(n_arr)

"""

Given array:
[[45.5 52.  10. ]
 [ 5.  50.  25. ]]

Replace all elements of array which are greater than 30. to 5.25
New array :

[[45.5 45.5 10. ]
 [ 5.  50.  25. ]]


In [ ]:
"""
def np_ffill(arr, axis):
    idx_shape = tuple([slice(None)] + [np.newaxis] * (len(arr.shape) - axis - 1))
    idx = np.where(~np.isnan(arr), np.arange(arr.shape[axis])[idx_shape], 0)
    np.maximum.accumulate(idx, axis=axis, out=idx)
    slc = [np.arange(k)[tuple([slice(None) if dim==i else np.newaxis
        for dim in range(len(arr.shape))])]
        for i, k in enumerate(arr.shape)]
    slc[axis] = idx
    return arr[tuple(slc)]


def random_array(shape):
    choices = [1, 2, 3, 4, np.nan]
    out = np.random.choice(choices, size=shape)
    return out

ra = random_array((1, 3, 8))
print('arr')
print(ra)
print('\nffull')
print(np_ffill(ra, 1))

"""

arr
[[[ 2.  2.  4.  3.  2.  3.  4.  1.]
  [nan  2.  1.  1.  2.  4.  1.  3.]
  [ 3.  4. nan  1.  1.  4.  3.  2.]]]

ffull
[[[2. 2. 4. 3. 2. 3. 4. 1.]
  [2. 2. 1. 1. 2. 4. 1. 3.]
  [3. 4. 1. 1. 1. 4. 3. 2.]]]




---



---



In [ ]:
!pip install pyvital vitaldb vital
!pip install neurokit2

In [ ]:

import vitaldb
import neurokit2 as nk
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [ ]:

Hemodynamic_Parameters = [
'Solar8000/ST_II',
'Solar8000/ST_VI',
'Solar8000/ART_DBP',
'Solar8000/ART_SBP'
]

In [ ]:
SRATE = 100
vals = vitaldb.load_case(1, Hemodynamic_Parameters, 1 / SRATE)

In [ ]:
%%time

#if  'Solar8000/ST_II' == nan, then set to 'Solar8000/ST_VI' (vise varsa)
for i in tqdm(range(vals.shape[0])):
  if np.isnan(vals[i][0]):
    vals[i][0] = vals[i][1]
  if np.isnan(vals[i][1]):
    vals[i][1] = vals[i][0]
  if np.isnan(vals[i][1]) and np.isnan(vals[i][0]):
    vals[i][0] = 0
    vals[i][1] = 0



#function to fill a nan value of a column with the previous value ('Solar8000/ART_DBP','Solar8000/ART_SBP')
def np_ffill(arr, axis):
    idx_shape = tuple([slice(None)] + [np.newaxis] * (len(arr.shape) - axis - 1))
    idx = np.where(~np.isnan(arr), np.arange(arr.shape[axis])[idx_shape], 0)
    np.maximum.accumulate(idx, axis=axis, out=idx)
    slc = [np.arange(k)[tuple([slice(None) if dim==i else np.newaxis
        for dim in range(len(arr.shape))])]
        for i, k in enumerate(arr.shape)]
    slc[axis] = idx
    return arr[tuple(slc)]



vals = np_ffill(vals,0)




MAP = []
indx=1
for i in tqdm(range(vals.shape[0])):
  MAP.append([(vals[i][3] + 2 * vals[i][2])/3,indx])
  indx+=1
vals = np.append(vals, MAP ,axis = 1)

  0%|          | 0/1103281 [00:00<?, ?it/s]

  0%|          | 0/1103281 [00:00<?, ?it/s]

CPU times: user 13.3 s, sys: 186 ms, total: 13.5 s
Wall time: 13.9 s


In [ ]:
vals.shape

(1103100, 6)

In [ ]:
vals = vals[~np.isnan(vals).any(axis=1)]

In [ ]:
for i in tqdm(range(vals.shape[0]-1)):
  print(vals[i+1][-1]-vals[i][-1])



---



---



In [ ]:
import pandas as pd

df = pd.DataFrame(vals)
df.columns = [
'Solar8000/ST_II',
'Solar8000/ST_VI',
'Solar8000/ART_DBP',
'Solar8000/ART_SBP',
'MAP',
'2_SEC_Time'
]


In [ ]:
df.to_csv("Patient_1.csv",index=False)

In [ ]:
df.shape

(1103281, 6)

In [ ]:
df.head()

,Solar8000/ST_II,Solar8000/ST_VI,Solar8000/ART_DBP,Solar8000/ART_SBP,MAP,2_SEC_Time
0,NaN,NaN,NaN,NaN,NaN,1.0
1,NaN,NaN,NaN,NaN,NaN,2.0
2,NaN,NaN,NaN,NaN,NaN,3.0
3,NaN,NaN,NaN,NaN,NaN,4.0
4,NaN,NaN,NaN,NaN,NaN,5.0


In [ ]:

df = df.query("MAP.notna()")

In [ ]:
df_trks = pd.read_csv('https://api.vitaldb.net/trks')  # read track list
df_cases = pd.read_csv("https://api.vitaldb.net/cases")  # read case information


In [ ]:
list(df_trks['tname'])

['BIS/BIS',
 'BIS/EEG1_WAV',
 'BIS/EEG2_WAV',
 'BIS/EMG',
 'BIS/SEF',
 'BIS/SQI',
 'BIS/SR',
 'BIS/TOTPOW',
 'Primus/AWP',
 'Primus/CO2',
 'Primus/COMPLIANCE',
 'Primus/ETCO2',
 'Primus/EXP_DES',
 'Primus/EXP_SEVO',
 'Primus/FEN2O',
 'Primus/FEO2',
 'Primus/FIN2O',
 'Primus/FIO2',
 'Primus/INCO2',
 'Primus/INSP_DES',
 'Primus/INSP_SEVO',
 'Primus/MAC',
 'Primus/MAWP_MBAR',
 'Primus/MV',
 'Primus/PAMB_MBAR',
 'Primus/PEEP_MBAR',
 'Primus/PIP_MBAR',
 'Primus/PPLAT_MBAR',
 'Primus/RR_CO2',
 'Primus/SET_AGE',
 'Primus/SET_FIO2',
 'Primus/SET_FRESH_FLOW',
 'Primus/SET_INSP_PAUSE',
 'Primus/SET_INSP_TM',
 'Primus/SET_INTER_PEEP',
 'Primus/SET_PIP',
 'Primus/SET_RR_IPPV',
 'Primus/SET_TV_L',
 'Primus/TV',
 'Primus/VENT_LEAK',
 'SNUADC/ART',
 'SNUADC/ECG_II',
 'SNUADC/ECG_V5',
 'SNUADC/PLETH',
 'Solar8000/ART_DBP',
 'Solar8000/ART_MBP',
 'Solar8000/ART_SBP',
 'Solar8000/BT',
 'Solar8000/ETCO2',
 'Solar8000/FEO2',
 'Solar8000/FIO2',
 'Solar8000/GAS2_EXPIRED',
 'Solar8000/GAS2_INSPIRED',
 'Solar